# <font color="red"> MBA em IA e Big Data</font>
## <span style="color:red">Redes Neurais e Deep Learning</span>
## **Exemplo de Uso de LSTM**

### **REDES NEURAIS RECORRENTES com**
### **VALIDAÇÃO CRUZADA**

### Profa. Roseli Ap. Francelin Romero

**PROBLEMA**: SERIE TEMPORAL - Previsão de passageiros de uma Cia Aérea Internacional.

Este é um problema em que, dados um ano e um mês, a tarefa é prever o número de passageiros (x 1.000) de companhias aéreas internacionais. Os dados variam de janeiro de 1949 a dezembro de 1960,  12 anos, com 144 observações.





Por default o Keras mantem um estado com um batch (stateful=True). Um batch é um no. de linhas do conjunto de dados usado para o treinamento. Entre uma camada e outra na LTSM o estado é desconsiderado (limpado) por default. Isto nos dá um controle mais fino entre os estados e podemos usar a função resetstates(). <\br>

**LSTM PARA REGRESSÃO** A cam. LSTM espera uma matriz como entrada: [samples, time steps, features], onde?

**Samples**: são as observações do dominio da aplicação, normalmente as linhas do dataset.</br> 
**Time steps**: são passos de tempo de separação de uma dada variavel para uma dada observação.</br>
**Features**: são medidas separadas observadas em cada tempo da observação.

Nesta série, os seguintes passos serão considerados:

- Carregar o dataset do CSV file.
- Transformar o dataset de modo a adequa-lo para o modelo LSTM, incluindo: 
- 1 - Transformar os dados para um problema de aprendizado supervisionado. 
- 2 - Transformar os dados para ser "stationary". 
- 3 - Transformar os dados para que fiquem no interv. [-1,1]. 
- 4 - Ajusta o modelo da rede LSTM para o conj. de treinamento. 
- 5 - Avaliar o modelo LSTM estático no conjunto de teste. 
- 6 - Visualizar o desempenho da Previsão. 

In [1]:

# LSTM for international airline passengers problem with regression framing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# converte um array de valores numa matriz de dados 
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):        
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

# as sementes ajudam a ter resultados reproduzíveis
from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(2)
# Carregar o dataset
dataframe = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')


Codigo com modificações para separar e rodar o conjunto de dados com o TimeSerieSplit


In [2]:
# reshape em X=t and Y=t+1
look_back = 1
X_dataset, Y_dataset = create_dataset(dataset, look_back)

print('X(t)', 'Y(t+1)')
for i in range(len(X_dataset)):
    print(X_dataset[i], Y_dataset[i])

X(t) Y(t+1)
[112.] 118.0
[118.] 132.0
[132.] 129.0
[129.] 121.0
[121.] 135.0
[135.] 148.0
[148.] 148.0
[148.] 136.0
[136.] 119.0
[119.] 104.0
[104.] 118.0
[118.] 115.0
[115.] 126.0
[126.] 141.0
[141.] 135.0
[135.] 125.0
[125.] 149.0
[149.] 170.0
[170.] 170.0
[170.] 158.0
[158.] 133.0
[133.] 114.0
[114.] 140.0
[140.] 145.0
[145.] 150.0
[150.] 178.0
[178.] 163.0
[163.] 172.0
[172.] 178.0
[178.] 199.0
[199.] 199.0
[199.] 184.0
[184.] 162.0
[162.] 146.0
[146.] 166.0
[166.] 171.0
[171.] 180.0
[180.] 193.0
[193.] 181.0
[181.] 183.0
[183.] 218.0
[218.] 230.0
[230.] 242.0
[242.] 209.0
[209.] 191.0
[191.] 172.0
[172.] 194.0
[194.] 196.0
[196.] 196.0
[196.] 236.0
[236.] 235.0
[235.] 229.0
[229.] 243.0
[243.] 264.0
[264.] 272.0
[272.] 237.0
[237.] 211.0
[211.] 180.0
[180.] 201.0
[201.] 204.0
[204.] 188.0
[188.] 235.0
[235.] 227.0
[227.] 234.0
[234.] 264.0
[264.] 302.0
[302.] 293.0
[293.] 259.0
[259.] 229.0
[229.] 203.0
[203.] 229.0
[229.] 242.0
[242.] 233.0
[233.] 267.0
[267.] 269.0
[269.] 270.0


In [3]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
i = 1
score = []
sum_Train_Error =0
sum_Test_Error = 0
for tr_index, val_index in tscv.split(X_dataset):
    X_tr, X_val = X_dataset[tr_index], X_dataset[val_index]
    y_tr, y_val = Y_dataset[tr_index], Y_dataset[val_index]

    scaler = MinMaxScaler(feature_range=(0, 1))
    #Fit apenas no conjunto de treino
    X_tr = scaler.fit_transform(X=X_tr)
    y_tr = scaler.transform(y_tr.reshape(-1, 1))
    #No conjunto de treino, apenas aplica o fit obtido no conjunto de treino 
    X_val = scaler.transform(X=X_val)
    y_val = scaler.transform(y_val.reshape(-1, 1))

    #print("TRAIN:", tr_index, "TEST:", val_index)
    #print("TrainX:", X_tr, "TrainY:", y_tr)
    # reshape entrada na forma [samples, time steps, features]
    trainX = np.reshape(X_tr, (X_tr.shape[0], 1, X_tr.shape[1]))
    testX = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
    # criando and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, y_tr, epochs=100, batch_size=1, verbose="auto")
    # model.score(testX,y_val)
    model.summary()
    # fazer as predicoes
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # inverter antes as predicoes
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform(y_tr)
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform(y_val)

    # calcular o RMSE error root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    sum_Train_Error += trainScore
    sum_Test_Error += testScore
    i += 1

print("Resultados")
print(" A media no Conjunto de Treinamento nos %d folds foi de %f" % (i-1, sum_Train_Error/i-1)) 
print(" A media no Conjunto de Test nos %d folds foi de %f" % (i-1, sum_Test_Error/i-1))  

2022-11-07 18:24:04.829206: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-11-07 18:24:05.209290: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
27/27 [==============================] - 2s 1ms/step - loss: 0.3404
Epoch 2/100
27/27 [==============================] - 0s 1ms/step - loss: 0.2988
Epoch 3/100
27/27 [==============================] - 0s 1ms/step - loss: 0.2639
Epoch 4/100
27/27 [==============================] - 0s 1ms/step - loss: 0.2343
Epoch 5/100
27/27 [==============================] - 0s 1ms/step - loss: 0.2087
Epoch 6/100
27/27 [==============================] - 0s 1ms/step - loss: 0.1862
Epoch 7/100
27/27 [==============================] - 0s 1ms/step - loss: 0.1675
Epoch 8/100
27/27 [==============================] - 0s 1ms/step - loss: 0.1488
Epoch 9/100
27/27 [==============================] - 0s 1ms/step - loss: 0.1337
Epoch 10/100
27/27 [==============================] - 0s 1ms/step - loss: 0.1191
Epoch 11/100
27/27 [==============================] - 0s 1ms/step - loss: 0.1077
Epoch 12/100
27/27 [==============================] - 0s 1ms/step - loss: 0.0962
Epoch 13/100
27/27 [=================

Podemos ver que em media obtemos um erro aproximado de 15 passageiros (em milhares) no conj. de treino e um erro aproximado de 30 pasageiros (em milhares) no conj. de teste.